<a href="https://colab.research.google.com/github/aytekin827/TIL/blob/main/search%20algorithm/%EC%B5%9C%EB%8B%A8%EA%B2%BD%EB%A1%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 최단경로

* **다익스트라 알고리즘**   
[원리]
 1. 출발노드를 설정한다.
 2. 최단 거리 테이블을 초기화한다.
 3. 방문하지 않은 노드 중에서 최단 거리가 가장 짧은 노드를 선택한다.
 4. 해당 노드를 거쳐 다른 노드로 가는 비용을 계산하여 최단 거리 테이블을 갱신한다. => **그리디 개념**
 5. 위 과정에서 3번과 4번을 반복한다.     

 [조건]  
 음의 간선이 없을때

 [활용]  
 GPS소프트웨어 기본 알고리즘

* 플로이드 워셜 알고리즘
